In [51]:
# notebook config

# Relative path to competition data export file
CSV = "data_export_m4-metal-medium.csv"

# Relative path to competition directory
COMPETITION_DIR = ".." 

# Competition dataset names
DATASETS = []

# Competition recall/ap threshold
RECALL_AP_THRESHOLD = 0.9

# Markdown template file
MARKDOWN = "./_latitude-m4-metal-medium.md"

In [52]:
# package imports

import pandas as pd
import os
import glob
import math
import re
import helper
import importlib
from IPython.core.display import HTML

In [53]:
# global config

pd.set_option('display.precision', 2)

In [54]:
# read CSV

df = pd.read_csv( CSV )


In [55]:
# get track/dataset groups

grps = df.groupby(["track","dataset"])

In [56]:
# transform columns

dfs = [] # accumulate individual track dataframes

# iterate groups
for name, group in grps:

    # extract groupby track and dataset
    track = name[0]
    dataset = name[1]
   
    # produce track ranking
    ranking_df = group[ group["recall/ap"]>=RECALL_AP_THRESHOLD ]\
        .groupby(["algorithm"]) \
        .max("qps") \
        .sort_values("qps", ascending=False) \
        [["qps","recall/ap"]] 
    
    # set 'status' column as 'qualified' for all surviving rows
    ranking_df['status'] = 'ok'

    # return the algorithm index as a column
    ranking_df.reset_index(inplace=True)
    #print(ranking_df)
    #print(list(ranking_df.index))

    # retrieve all participating track algorithm names via track algo subdirectory 
    track_dir = os.path.join( COMPETITION_DIR, "%s/*/Dockerfile" % track )
    algos_participating = [os.path.basename(os.path.dirname(p)) for p in glob.glob( track_dir ) ]
  
    # compute difference of algo lists of track subdirs and algos in results - these did not qualify
    algos_did_not_qualify = list( set(algos_participating) - set(list(ranking_df["algorithm"])) )

    # append not-qualified algos to dataframe
    for algo in algos_did_not_qualify:
        ranking_df = pd.concat([ranking_df, pd.DataFrame([{'algorithm':algo,'status':'error'}])], ignore_index = True)

    # create a rank numeric column
    ranking_df['rank'] = ranking_df.apply( lambda row: int(row.name)+1, axis=1)
    ranking_df['rank'] = ranking_df['rank'].astype('Int64') # Int64 supports NaN

    # change 'qps' to K
    ranking_df['qps'] = ranking_df['qps']/1000.0
    
    # return the track and dataset as column
    ranking_df['track'] = track
    ranking_df['dataset'] = dataset
    #print(ranking_df)

    dfs.append(ranking_df)


In [57]:
# combine all track dataframes
master_df = pd.concat(dfs)
master_df

,algorithm,qps,recall/ap,status,rank,track,dataset
0,zilliz,213.29,0.93,ok,1,filter,yfcc-10M
1,pinecone,146.72,0.92,ok,2,filter,yfcc-10M
2,puck,62.26,0.94,ok,3,filter,yfcc-10M
3,parlayivf,55.03,0.95,ok,4,filter,yfcc-10M
4,wm_filter,20.87,0.92,ok,5,filter,yfcc-10M
5,pyanns,8.99,0.91,ok,6,filter,yfcc-10M
6,faissplus,8.49,0.91,ok,7,filter,yfcc-10M
7,faiss,7.33,0.99,ok,8,filter,yfcc-10M
8,cufe,6.32,0.94,ok,9,filter,yfcc-10M
9,hwtl_sdu_anns_filter,NaN,NaN,error,10,filter,yfcc-10M


In [62]:
# pivot via 'rank' so that track rankings are parallel across columns

pivot_df =  master_df.pivot_table(index=['rank'], 
                                  columns=['track'], 
                                  values=['algorithm','qps','status'],
                                  aggfunc='first')
#pivot_df


In [63]:
# group specific track columns

reorder_df = pivot_df[ [ ('algorithm','filter'), ( 'qps','filter' ), \
            ('algorithm','sparse'), ( 'qps','sparse' ), \
            ('algorithm','ood'),    ( 'qps','ood' ) ] ]
#reorder_df

In [64]:
# swap hierarchical index for columns

swap_df = reorder_df.swaplevel(0,1, axis=1)
#swap_df

In [65]:
# add per track 'rank' column ensuring no rank to algorithms that did not qualify

for track in master_df['track'].unique():
    qps = list( swap_df[ (track,'qps') ] )
    track_ranking = map(lambda el: float('nan') if math.isnan(qps[el-1]) else el,
                  range(1,len(qps)+1) )
    #print(track_ranking, list(ranking))
    col_to_find = (track, 'algorithm')
    col_idx = list(swap_df.columns).index(col_to_find)

    swap_df.insert(col_idx, (track,'rank'), list(track_ranking) )
    swap_df[(track,'rank')] = swap_df[(track,'rank')].astype('Int64')
    
#swap_df

In [66]:
# style the dataframe (getting ready for html export)

display_df_styled = swap_df.style
display_df_styled

In [67]:
# convert styled table to html string

importlib.reload(helper) # need to reload if editing simultaneuosly

# hide row index and export to html
html = display_df_styled.\
    format(formatter="{:20,.1f}K", na_rep="", \
           subset=pd.IndexSlice[:, pd.IndexSlice['filter','qps']]).\
    format(formatter="{:20,.1f}K", na_rep="", \
           subset=pd.IndexSlice[:, pd.IndexSlice['sparse','qps']]).\
    format(formatter="{:20,.1f}K", na_rep="", \
           subset=pd.IndexSlice[:, pd.IndexSlice['ood','qps']]).\
    hide(axis=0).\
    to_html()

# remove style tag (markdown doesn't use it anyway)
new_html = helper.remove_style_prefix(html)

# replace all "nan"s with empty string
new_html = new_html.replace("nan","")

# insert links for specific use cases (ie, errors)
new_html = helper.replace_table_with_links(new_html, swap_df)

# display in-line
HTML(new_html)

In [69]:
# insert html into the markdown

with open(MARKDOWN,'r') as md:
    contents = md.read()
    
# locate insertion point via regex
regexpr = "(## Results\n)"
matches = re.search(regexpr, contents, re.M)

# insert table html
new_contents = contents[0:matches.span(0)[1]] + new_html +" \n" + contents[matches.span(0)[1]:]
#print(new_contents[0:1000])

# update the markdown
fname = os.path.join( "..", MARKDOWN.replace("_","") )
with open( fname, 'w') as md:
    contents = md.write(new_contents)

print("Wrote", fname)

Wrote .././latitude-m4-metal-medium.md
